# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd

# Import API key
from api_keys import geoapify_key
from shapely.geometry import Point


In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
geometry = [Point(xy) for xy in zip(city_data_df['Lng'], city_data_df['Lat'])]
geo_city_data_df = gpd.GeoDataFrame(city_data_df, geometry=geometry, crs="EPSG:4326")

# Display the map
# YOUR CODE HERE
plot = geo_city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    color='Humidity',
    size='Humidity',
    hover_cols=['City', 'Humidity'],
    title='Cities with Points Sized by Humidity'
)

plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_df= city_data_df[(city_data_df["Humidity"] <= 40 ) & (city_data_df["Humidity"] >= 30)]

# Drop any rows with null values
# YOUR CODE HERE
city_data_df = city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
city_data_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
radius = 100000000

params = { 
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distance lt {radius} and @location = {lng},{lat}"
    params["bias"] = f"@location={lng},{lat}"
    params["categories"] = "accommodation.hotel" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
howard springs - nearest hotel: No hotel found
vilyuysk - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
prince george - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
orkney - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
college - nearest hotel: No hotel found
jabal os saraj - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
bentiu - nearest hotel: No hotel found
dibba al-hisn - nearest hotel: No hotel found
nurota - nearest hotel: No hotel found
ghanzi - nearest hotel: No hotel found
yanykurgan - nearest hotel: No hotel found
valley city - nearest hotel: No hotel found
el abiodh sidi cheikh - nearest hotel: No hotel found
hoopstad - nearest hotel: No hotel found
middelburg - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
badger - nearest hotel: No

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,20,howard springs,-12.4970,131.0470,295.83,31,5,5.15,AU,1684715437,No hotel found
36,36,vilyuysk,63.7553,121.6247,276.30,35,0,4.68,RU,1684715447,No hotel found
66,66,carnarvon,-24.8667,113.6333,293.68,39,77,8.07,AU,1684715454,No hotel found
76,76,klyuchi,52.2667,79.1667,288.45,39,35,2.60,RU,1684715457,No hotel found
89,89,prince george,53.9166,-122.7530,292.84,37,75,9.77,CA,1684715416,No hotel found
96,96,whitehorse,60.7161,-135.0538,290.58,31,75,4.12,CA,1684715464,No hotel found
122,122,orkney,-26.9802,26.6727,282.52,31,1,1.26,ZA,1684715475,No hotel found
127,127,kapuskasing,49.4169,-82.4331,282.20,37,20,3.09,CA,1684715478,No hotel found
145,145,college,64.8569,-147.8028,295.12,39,75,2.06,US,1684715396,No hotel found
147,147,jabal os saraj,35.1183,69.2378,293.28,40,100,0.97,AF,1684715492,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
geometry = [Point(xy) for xy in zip(hotel_df['Lng'], hotel_df['Lat'])]
geo_city_data_df = gpd.GeoDataFrame(city_data_df, geometry=geometry, crs="EPSG:4326")

# Display the map
# YOUR CODE HERE
plot = geo_city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    color='Humidity',
    size='Humidity',
    hover_cols=['City', 'Humidity', 'Hotel Name'],
    title='Cities with Points Sized by Humidity'
    )

plot